In [7]:
# README.md

# Mail Client Library for Python

This project aims to provide a simple and easy to use library for sending emails from Python. The project is still in an early stage, but it should be usable for most of the common use cases.

Main idea is to provide a simple API for sending emails, but still allow for customization, if needed.
Use factory methods to create instances of `MailClient` class. The factory methods are `MailClient.create_gmail_client` and `MailClient.create_smtp_client`. Also you can use `MailClient.create_client` method, which will create a client based on the provided configuration.

**Structure:**
- Abstract class `AbstractMailClient` defines the interface for sending, receiving, parsing, filtering, email and their grouping entities: inbox, thread, message with attachments. Specifies the base filters like unread, starred, etc.
- `SimpleGmailClient` class implements the interface for sending emails using `simplegmail` package using credentials provided in form of `json` files.
- `SMTPClient` class implements the interface for sending emails using SMTP protocol.
- `ImapClient` class implements the interface for receiving emails using IMAP protocol.
- `Pop3Client` class implements the interface for receiving emails using POP3 protocol.
  
**Project/Package Structure:**
- `mail_client` - main package
  - `mail_client.py` - contains `MailClient` class, which is the main class of the package
  - `abstract_mail_client.py` - contains `AbstractMailClient` class, which defines the interface for sending, receiving, parsing, filtering, email and their grouping entities: inbox, thread, message with attachments. Specifies the base filters like unread, starred, etc.
  - `simple_gmail_client.py` - contains `SimpleGmailClient` class, which implements the interface for sending emails using `simplegmail` package using credentials provided in form of `json` files.
  - `smtp_client.py` - contains `SMTPClient` class, which implements the interface for sending emails using SMTP protocol.
  - `imap_client.py` - contains `ImapClient` class, which implements the interface for receiving emails using IMAP protocol.
  - `pop3_client.py` - contains `Pop3Client` class, which implements the interface for receiving emails using POP3 protocol.
  - `exceptions.py` - contains custom exceptions
  - `utils.py` - contains utility functions
**Notebook methodology:**
  - each client has its own cell with examples and tests for its methods
  - every cell is registed in the Factory class which is the first code cell in the notebook. This allows for easy access to the cells and their execution and wiring of the cells in the notebook and cross notebooks.

In [8]:
# Installation and Setup
%pip install simplegmailclient

ERROR: Could not find a version that satisfies the requirement simplegmailclient (from versions: none)
ERROR: No matching distribution found for simplegmailclient
Note: you may need to restart the kernel to use updated packages.


In [9]:
# Configuration Partials
%env MAIL_CLIENT_WORKING_DIR=.
%env MAIL_CLIENT_DATA_DIR=.
%env MAIL_CLIENT_MODULE_NAME=mail_client
%env MAIL_CLIENT_INTERFACE=MailClient
%env MAIL_CLIENT_CLASS=SimpleGmailClient
%env MAIL_CLIENT_CONFIG_FILE_PATH=mail_client_config.json
%env MAIL_CLIENT_SECRET_FILE_PATH=mail_client_credentials.json
%env MAIL_CLIENT_CREDENTIALS_FILE_PATH=mail_client_credentials.json


env: MAIL_CLIENT_WORKING_DIR=.
env: MAIL_CLIENT_DATA_DIR=.
env: MAIL_CLIENT_MODULE_NAME=mail_client
env: MAIL_CLIENT_INTERFACE=MailClient
env: MAIL_CLIENT_CLASS=SimpleGmailClient
env: MAIL_CLIENT_CONFIG_FILE_PATH=mail_client_config.json
env: MAIL_CLIENT_SECRET_FILE_PATH=mail_client_credentials.json
env: MAIL_CLIENT_CREDENTIALS_FILE_PATH=mail_client_credentials.json


In [10]:
from pathlib import Path
import abc.ABCMeta, types, ABC

class AbstractMailClient(ABC):
    """ Abstract class for mail client"""
    
    
    ...
class MailClient(AbstractMailClient):
    """ Main class for mail client"""
    def __init__(self, **kwargs):
        ...
    @staticmethod
    def create_client(**kwargs):
        ...
    @staticmethod
    def create_gmail_client(**kwargs):
        ...
    @staticmethod
    def create_smtp_client(**kwargs):
        ...
    @staticmethod
    def create_imap_client(**kwargs):
        ...
    @staticmethod
    def create_pop3_client(**kwargs):
        ...
    @staticmethod
    def create_client_from_config(**kwargs):
        ...
    @staticmethod
    def create_gmail_client_from_config(**kwargs):
        ...
    @staticmethod
    def create_smtp_client_from_config(**kwargs):
        ...
    @staticmethod
    def create_imap_client_from_config(**kwargs):
        ...
    @staticmethod
    def create_pop3_client_from_config(**kwargs):
        ...
    ...
class SimpleGmailClient(MailClient):
    """ Simple Gmail client using simplegmail package. Takes credentials in form of json files
    Implements a simple Gmail client with the following methods:
    - send_message - sends an email message 
    - get_messages - returns all messages matching the optional filter
    - get_inbox - returns all messages in the inbox
    
    Example:
    ```python
    ### *SendMessage, Invoke client from config file*
    from mail_client import MailClient
    class SimpleGmailClient(MailClient):
        ...
        
    class GmailCredentials(object):
        def __init__(self, credentials_file_path: Path, secret_file_path: Path):
            self.credentials_file_path = credentials_file_path
            self.secret_file_path = secret_file_path
        def invoke(self):
            client = MailClient.create_gmail_client(credentials_file_path=self.credentials_file_path, secret_file_path=self.secret_file_path)
            client.send_message(...)
            client.get_messages(query="from:helpdesk.perplexity.ai")
    mclient: SimpleGmailClient = MailClient.create_client_from_config(GmailCredentials(credentials_file_path='credentials.json', secret_file_path='secret.json'))   
            
    ### *GetMessages*  
    client = MailClient.create_gmail_client(credentials_file_path='credentials.json', secret_file_path='secret.json')
    client.send_message(...)
    Get messages by filter (unread, starred, etc.):
    from mail_client import MailClient
    client = MailClient.create_gmail_client()
    messages = client.get_messages(filter='unread')
    Get all messages in the inbox:
    from mail_client import MailClient
    client = MailClient.create_gmail_client()
    messages = client.get_inbox(name='inbox')
    
    ```
    """
    engine = 'simplegmail.Gmail'
    working_dir: Path = "."
    credentials_file_path: Path = working_dir / "credentials.json"
    secret_file_path: Path = working_dir / "secret.json"
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._engine = 'simplegmail.Gmail'
        self._client = None
        self._credentials_file_path = kwargs.get('credentials_file_path', None)
        self._secret_file_path = kwargs.get('secret_file_path', None)
        self._email = kwargs.get('email', None)
        self._email_address = kwargs.get('email_address', None)
        self._email_address_name = kwargs.get('email_address_name', None)
        self._name = kwargs.get('name', None)
        self._client = self._create_client()
    def _create_client(self):
        if self._client is None:
            self._client = self._create_client_from_config()
        return self._client
    def _create_client_from_config(self):
        if self._client is None:
            self._client = self.create_client_from_config(credentials_file_path=self._credentials_file_path, secret_file_path=self._secret_file_path)
        return self._client
    @staticmethod
    def create_client(**kwargs):
        client = SimpleGmailClient(**kwargs)
        return client
    def _get_messages(self, **kwargs):
        messages = self._client.get_messages(**kwargs)
        return messages
    def _get_inbox(self, **kwargs):
        messages = self._client.get_inbox(**kwargs)
        return messages
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._engine = 'simplegmail.Gmail'
        self._client = None
        self._credentials_file_path = kwargs.get('credentials_file_path', None)
        self._secret_file_path = kwargs.get('secret_file_path', None)
        self._email = kwargs.get('email', None)
        self._email_address = kwargs.get('email_address', None)
        self._email_address_name = kwargs.get('email_address_name', None)
        self._name = kwargs.get('name', None)
        self._client = self._create_client()

    

    
    

ModuleNotFoundError: No module named 'abc.ABCMeta'; 'abc' is not a package

In [ ]:
# Factory.py

import importlib
import os
import sys
from typing import List, Dict, Tuple, Callable, Any, Union, Optional
import inspect, abc, ABCMeta, types
class NotebookHelper(object):
    cells: Dict[str, Callable] = {}
    @staticmethod
    def register_cell(name: str, cell: Callable):
        NotebookHelper.cells[name] = cell
    @staticmethod
    def get_cell(name: str) -> Callable:
        return NotebookHelper.cells[name]
    @staticmethod
    def get_cells(self) -> Dict[str, Callable]:
        return NotebookHelper.cells
    @staticmethod
    def get_all_cells() -> List[Callable]:
        return list(NotebookHelper.cells.values())
    @staticmethod
    def execute_cell(name: str):
        NotebookHelper.cells[name]()
    @staticmethod
    def execute_all_cells():
        for cell in NotebookHelper.get_all_cells():
            cell()
    @staticmethod
    def create_notebook_cell(name: str, cell: Callable):
        NotebookHelper.register_cell(name, cell)
        return cell
    

In [12]:
from abc import ABC, abstractmethod
from simplegmail import Gmail
from simplegmail.message import Message
from simplegmail.attachment import Attachment

class MailClient(ABC):
    """Abstract Mail Client Class"""
    
    @abstractmethod
    def send_message(self, message):
        pass

    @abstractmethod
    def get_messages(self, **kwargs):
        pass
    

class SimpleGmailClient(MailClient):
    """Gmail Client Implementation using `simplegmail`"""
    
    def __init__(self, client):
        self.client = client

    def send_message(self, message):
        # Implementation of sending message using `simplegmail`
        pass

    def get_messages(self, filter: str = None, **kwargs) -> list[Message]:
        """Get messages from Gmail inbox with optional filter like from:email, to:email, etc."""
        # Implementation of receiving message using `simplegmail`
        _ = self.client.get_messages(filter, **kwargs)
        return _

def client_factory(client_type, credentials: dict = None, **kwargs) -> MailClient:
    """Factory function to create mail clients"""
    
    if client_type == 'gmail':
        if credentials is None:
            return SimpleGmailClient(Gmail())
        else: return SimpleGmailClient(Gmail(*credentials))
    else:
        raise ValueError("Invalid client type")
if __name__ == '__main__':
    secret_contents = '{"installed":{"client_id":"543382583890-fktcl5fttumda3ph3ts4lio61ug2e3vv.apps.googleusercontent.com","project_id":"absolute-origin-378108","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"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","redirect_uris":["http://localhost"]}}'
    credentials_contents = '{"access_token": "ya29.a0AfB_byDbRoIHYM8ca14ui9kp3MLebOwuVclhv0iO7nblAGnGBAeaI36e5zlEG4BG_hZRfJ30qreXSfSRY3HBsPffCqg-VKY-37q_uWpf7dHlbgBlClb4jzGjyUasZiiCYCE6LhIMd04cj-xMTS3QyhAnG5iZo9CTqA-E5waCgYKAZcSARESFQHGX2MiaQ39FNj-T1ussLXoosXckQ0173", "client_id": "543382583890-fktcl5fttumda3ph3ts4lio61ug2e3vv.apps.googleusercontent.com", "client_secret": "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", "refresh_token": "1//05_8a7sKw86sZCgYIARAAGAUSNwF-L9IrnbU_BYRqP9w3GWSOIgYkf6cGkhIFvdfmS_-unp-65cvo_baFcZKtz2WFsJeexcrBKwg", "token_expiry": "2023-12-29T20:43:36Z", "token_uri": "https://oauth2.googleapis.com/token", "user_agent": null, "revoke_uri": "https://oauth2.googleapis.com/revoke", "id_token": null, "id_token_jwt": null, "token_response": {"access_token": "ya29.a0AfB_byDbRoIHYM8ca14ui9kp3MLebOwuVclhv0iO7nblAGnGBAeaI36e5zlEG4BG_hZRfJ30qreXSfSRY3HBsPffCqg-VKY-37q_uWpf7dHlbgBlClb4jzGjyUasZiiCYCE6LhIMd04cj-xMTS3QyhAnG5iZo9CTqA-E5waCgYKAZcSARESFQHGX2MiaQ39FNj-T1ussLXoosXckQ0173", "expires_in": 3599, "scope": "https://www.googleapis.com/auth/gmail.settings.basic https://www.googleapis.com/auth/gmail.modify", "token_type": "Bearer"}, "scopes": ["https://www.googleapis.com/auth/gmail.modify", "https://www.googleapis.com/auth/gmail.settings.basic"], "token_info_uri": "https://oauth2.googleapis.com/tokeninfo", "invalid": false, "_class": "OAuth2Credentials", "_module": "oauth2client.client"}'
    with open('./client_secret.json', 'w') as f:
        f.write(secret_contents)
    with open('./credentials.json', 'w') as f:
        f.write(credentials_contents)
        
    credentials = {
        'credentials_file_path': 'credentials.json',
        'secret_file_path': 'client_secret.json'}
    client = client_factory('gmail', credentials)
    messages = client.get_messages(filter='from:helpdesk.perplexity.ai')
    print(messages)

/Users/pv/Documents/zbst.tech/perplexityai/.venv/lib/python3.10/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access secret_file_path: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


FileNotFoundError: Your 'client_secret.json' file is nonexistent. Make sure the file is in the root directory of your application. If you don't have a client secrets file, go to https://developers.google.com/gmail/api/quickstart/python, and follow the instructions listed there.

In [2]:
import json
import requests
import requests_oauthlib
from requests_oauthlib import OAuth2
from requests_oauthlib.oauth2_session import OAuth2Session
from perplexity import Perplexity
USER_AGENT = "Ask/2.4.1/224 (iOS; iPhone; Version 17.1) isiOSOnMac/false"
X_CLIENT_NAME = "Perplexity-iOS"
PPLX_PRO_SESSION={
    "ormwish@gmail.com": {
        "__cf_bm": "KUKzykfqXRH8NQ3dgSEYkdEEUGtnwLJ3VfVWo2mb.6M-1703866466-1-Ad08roLUpt+pxD+cj93Fwk5OqJL/6Uxg9smnh+jcEvnQevH3KvzOZUcVYD1VXIbZDF72NR9E6FvGRe/+E7URPng=", "AWSALB": "5moWd7YeazMK4ESR3u5JbbQSM1axN+qOOuLsluSdEh/F11ZOuCZrk70xliNVwiBALKLSoDMCzYR3A2CkVANjlmmGU71lXAI9bg0mARTlJabMPYsvYE8HEkvnv5gx", "AWSALBCORS": "5moWd7YeazMK4ESR3u5JbbQSM1axN+qOOuLsluSdEh/F11ZOuCZrk70xliNVwiBALKLSoDMCzYR3A2CkVANjlmmGU71lXAI9bg0mARTlJabMPYsvYE8HEkvnv5gx",
        "__Secure-next-auth.callback-url": "https%3A%2F%2Fwww.perplexity.ai%2Fapi%2Fauth%2Fsignin-callback%3Fredirect%3Dhttps%253A%252F%252Fwww.perplexity.ai", 
        "__cflb": "02DiuDyvFMmK5p9jVbWbam6CcSLCt41haN4sV1nYVegLL", 
        "next-auth.csrf-token": "eb01a25093ca02601a5d4579510fef45c35eb76f1cb63ad02fd08607da169632%7C0fdaca48bd5c542087a3eea29804c7e572126d2e079fd7e3fd4274b896cc2f77"}}
# peprlexity.ai AUTH uris
singin_email_uri="https://www.perplexity.ai/api/auth/signin-email"
singin_token_uri="https://www.perplexity.ai/api/auth/signin-token"

session = requests.Session()
session.cookies.update(PPLX_PRO_SESSION["ormwish@gmail.com"])
session.headers.update(
    {'User-Agent': USER_AGENT, 
     'X-Client-Name': X_CLIENT_NAME}
)

with open(".perplexity_session", "w") as f:
    f.write(json.dumps(PPLX_PRO_SESSION))

perplexity = Perplexity()
perplexity.search("How to deal with lamers?")

IndentationError: expected an indented block after function definition on line 16 (utils.py, line 16)